In [1]:
#!pip install pybboxes
#!pip install ultralytics

In [2]:
import os
import glob
import cv2
import pybboxes as pbx
from ultralytics import YOLO

In [3]:
det_model=YOLO("yolo.pt")#https://docs.ultralytics.com/models/yolov8/#performance-metrics"

In [4]:
det_model.info()

Model summary: 365 layers, 43631382 parameters, 0 gradients, 165.4 GFLOPs


(365, 43631382, 0, 165.40738559999997)

In [5]:
results = det_model('./orginal_images/000000190.png')


image 1/1 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\000000190.png: 704x1088 1 license, 3336.6ms
Speed: 23.0ms preprocess, 3336.6ms inference, 41.0ms postprocess per image at shape (1, 3, 704, 1088)


In [6]:
det_model(source='orginal_images/',
            save=False,
            save_txt=True,
            conf=0.1,
            device='cpu',
            project="object",
            name="pred")



image 1/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\000000190.png: 704x1088 1 license, 2740.0ms
image 2/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\000009372.png: 704x1088 1 license, 3297.4ms
image 3/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image265.jpg: 704x1088 3 licenses, 3584.2ms
image 4/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image266.jpg: 704x1088 3 licenses, 3278.6ms
image 5/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image267.jpg: 704x1088 3 licenses, 3282.7ms
image 6/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image268.jpg: 704x1088 5 licenses, 3325.9ms
image 7/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image269.jpg: 704x1088 5 licenses, 3441.2ms
image 8/35 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image270.jpg: 704x1

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'license', 1: 'face'}
 orig_img: array([[[198, 178, 147],
         [198, 178, 147],
         [199, 179, 148],
         ...,
         [149, 154, 163],
         [151, 156, 165],
         [155, 160, 169]],
 
        [[201, 181, 150],
         [201, 181, 150],
         [201, 181, 150],
         ...,
         [150, 155, 164],
         [155, 160, 169],
         [161, 166, 175]],
 
        [[204, 183, 155],
         [204, 183, 155],
         [203, 183, 152],
         ...,
         [161, 163, 173],
         [156, 159, 167],
         [151, 154, 162]],
 
        ...,
 
        [[100, 111, 143],
         [ 99, 110, 142],
         [ 98, 109, 141],
         ...,
         [ 59,  99, 127],
         [ 58,  98, 127],
         [ 57,  99, 128]],
 
        [[ 98, 111, 143],
         [ 97, 110, 142],
         [ 96, 109, 141],
         ...,
         [ 60, 10

In [7]:
images = sorted(glob.glob("orginal_images/"+"/*"))
os.mkdir("annot_txt")

text_dir=f'./object/pred/labels/'

try:
    for file in os.listdir(text_dir):
        if (file.endswith('.txt')):
            #frame_num = int(file.replace(".txt","").split("_")[1])
            with open(text_dir+file, 'r') as fin:
                for line in fin.readlines():
                    line = [float(item) for item in line.split()[1:]]
                    line = pbx.convert_bbox(line, from_type="yolo", to_type="voc", image_size=(1920, 1208))
                    data_string = " ".join(str(num) for num in line)
                    with open(f"annot_txt/{os.path.basename(file)}", "a") as f:
                        f.write(data_string+"\n")
except:
    print(f'{os.path.basename(file)} has no detected objects.')


In [8]:
def anonymize(image, regions):
    """
    Blurs the image, given the x1,y1,x2,y2 cordinates using Gaussian Blur.
    """
    for region in regions:
        x1,y1,x2,y2 = region
        x1, y1, x2, y2 = round(x1), round(y1), round(x2), round(y2)
        roi = image[y1:y2, x1:x2]
        blurred_roi = cv2.GaussianBlur(roi, (31, 31), 0)
        image[y1:y2, x1:x2] = blurred_roi
    return image
txt_folder = 'annot_txt/'
image_folder = 'orginal_images/'
output_folder = 'blur_image/'

In [17]:
# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all text files in the 'dir' folder
txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

for txt_file in txt_files:
    # Read the text file containing bounding box information
    with open(os.path.join(txt_folder, txt_file), 'r') as f:
        lines = f.readlines()

    # Extract bounding box coordinates from the txt file
    bboxes = []
    for line in lines:
        values = line.strip().split()
        x_min, y_min, x_max, y_max = map(int, values)  # Assuming VOC format with x_min, y_min, x_max, y_max
        bboxes.append([x_min, y_min, x_max, y_max])

    # Read the corresponding image
    image_file = txt_file.replace('.txt', '.jpg')  # Assuming image files have .jpg extension
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    # Apply Gaussian blur to each bounding box region
    for bbox in bboxes:
        image = anonymize(image, bboxes)

    # Save the blurred image to the output folder
    output_file = txt_file.replace('.txt', '.jpg')
    output_path = os.path.join(output_folder, output_file)
    cv2.imwrite(output_path, image)

print("Task Completed,HAppy coding:)")

Task Completed,HAppy coding:)
